In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import pickle

In [12]:
data = pd.read_csv('hospital_supplier_large_dataset.csv')
data.head()

,Order_ID,Medicine_Name,Supplier_ID,Urgency,Distance_from_Supplier_km,Traffic_Delay_minutes,Stock_Availability,Affordability,Quality_Rating,Prior_Requests,Lead_Time_days,Supplier_Reliability,Contract_Existence,Inventory_Level_Hospital,Risk_of_Stockout,Emergency_Situation,Environmental_Factors,Optimal_Supplier
0,1,Insulin,S1_1,Low,476,37,Low,Affordable,2,15,1,0.89,No,64,0.96,Yes,Temperature-sensitive,Yes
1,1,Insulin,S1_2,Low,704,42,Low,Expensive,3,1,7,0.49,Yes,146,0.87,Yes,Temperature-sensitive,No
2,1,Insulin,S1_3,High,618,102,Available,Affordable,4,15,15,0.65,Yes,401,0.73,Yes,Temperature-sensitive,No
3,1,Insulin,S1_4,High,501,73,Available,Expensive,1,0,2,0.90,No,165,0.97,No,NaN,Yes
4,1,Insulin,S1_5,Medium,124,28,Available,Affordable,3,17,13,0.69,No,8,0.26,No,NaN,No


In [13]:
label_encoders = {}
for col in ['Urgency', 'Stock_Availability', 'Affordability', 'Contract_Existence',
            'Emergency_Situation', 'Environmental_Factors', 'Optimal_Supplier']:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le  # Store label encoder for each column

In [14]:
X = data.drop(columns=['Order_ID', 'Supplier_ID', 'Optimal_Supplier', 'Medicine_Name'])
y = data['Optimal_Supplier']

scaler = StandardScaler()
X = scaler.fit_transform(X)

y = to_categorical(y, num_classes=2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
model = Sequential([
    Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(2, activation='sigmoid')  # Output layer for binary classification
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
50/50 [==============================] - 1s 6ms/step - loss: 0.5872 - accuracy: 0.6794 - val_loss: 0.3690 - val_accuracy: 0.8625
Epoch 2/20
50/50 [==============================] - 0s 3ms/step - loss: 0.2876 - accuracy: 0.9006 - val_loss: 0.3052 - val_accuracy: 0.8625
Epoch 3/20
50/50 [==============================] - 0s 3ms/step - loss: 0.2373 - accuracy: 0.9044 - val_loss: 0.2503 - val_accuracy: 0.8675
Epoch 4/20
50/50 [==============================] - 0s 3ms/step - loss: 0.1991 - accuracy: 0.9137 - val_loss: 0.2099 - val_accuracy: 0.8800
Epoch 5/20
50/50 [==============================] - 0s 3ms/step - loss: 0.1660 - accuracy: 0.9244 - val_loss: 0.1769 - val_accuracy: 0.9050
Epoch 6/20
50/50 [==============================] - 0s 3ms/step - loss: 0.1597 - accuracy: 0.9212 - val_loss: 0.1639 - val_accuracy: 0.9175
Epoch 7/20
50/50 [==============================] - 0s 3ms/step - loss: 0.1335 - accuracy: 0.9419 - val_loss: 0.1356 - val_accuracy: 0.9525
Epoch 8/20
50/50 [==

In [6]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

16/16 [==============================] - 0s 1ms/step - loss: 0.0653 - accuracy: 0.9760
Test Loss: 0.06530769914388657, Test Accuracy: 0.9760000109672546


In [7]:
model.save('model.keras')

In [9]:
custom_data = [
    "High", 120, 15, "Available", "Affordable", 4, 3, 4, 0.9, "Yes", 150, 0.2, "Yes", np.nan
]

# Encode categorical values using the same label encoders used during training
custom_data_encoded = custom_data.copy()
custom_data_encoded[0] = label_encoders["Urgency"].transform([custom_data[0]])[0]
custom_data_encoded[3] = label_encoders["Stock_Availability"].transform([custom_data[3]])[0]
custom_data_encoded[4] = label_encoders["Affordability"].transform([custom_data[4]])[0]
custom_data_encoded[9] = label_encoders["Contract_Existence"].transform([custom_data[9]])[0]
custom_data_encoded[12] = label_encoders["Emergency_Situation"].transform([custom_data[12]])[0]
custom_data_encoded[13] = label_encoders["Environmental_Factors"].transform([custom_data[13]])[0]

# Convert to numeric array and scale numerical features
custom_data_encoded = np.array(custom_data_encoded, dtype=float).reshape(1, -1)
custom_data_scaled = scaler.transform(custom_data_encoded)

# Predict with the model
prediction = model.predict(custom_data_scaled)
optimal_supplier = np.argmax(prediction)  # Get the class with the highest probability

# Decode the prediction to a human-readable label if needed
optimal_supplier_label = "Optimal" if optimal_supplier == 1 else "Not Optimal"

print(f"Predicted Class: {optimal_supplier_label}")
print(f"Prediction Probabilities: {prediction}")


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
Predicted Class: Optimal
Prediction Probabilities: [[0.35128504 0.48970112]]


In [10]:
custom_data = [
    "Medium", 50, 20, "Low", "Affordable", 2, 9, 10, 0.75, "No", 250, 0.8, "No", np.nan
]

custom_data_encoded = custom_data.copy()
custom_data_encoded[0] = label_encoders["Urgency"].transform([custom_data[0]])[0]
custom_data_encoded[3] = label_encoders["Stock_Availability"].transform([custom_data[3]])[0]
custom_data_encoded[4] = label_encoders["Affordability"].transform([custom_data[4]])[0]
custom_data_encoded[9] = label_encoders["Contract_Existence"].transform([custom_data[9]])[0]
custom_data_encoded[12] = label_encoders["Emergency_Situation"].transform([custom_data[12]])[0]
custom_data_encoded[13] = label_encoders["Environmental_Factors"].transform([custom_data[13]])[0]

# Convert to numeric array and scale numerical features
custom_data_encoded = np.array(custom_data_encoded, dtype=float).reshape(1, -1)
custom_data_scaled = scaler.transform(custom_data_encoded)

# Predict with the model
prediction = model.predict(custom_data_scaled)
optimal_supplier = np.argmax(prediction)  # Get the class with the highest probability

# Decode the prediction to a human-readable label if needed
optimal_supplier_label = "Optimal" if optimal_supplier == 1 else "Not Optimal"

print(f"Predicted Class: {optimal_supplier_label}")
print(f"Prediction Probabilities: {prediction}")

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Predicted Class: Not Optimal
Prediction Probabilities: [[0.98349464 0.00823233]]
